In [130]:
import sympy as smp
import numpy as np
from IPython.display import Math, Latex

In [131]:
s, z = smp.symbols("s, z")
Ko_num = smp.Poly([0, 0, 0.255, 0.134], z)
Ko_den = smp.Poly([25, -28.83, 3.38], z)

In [132]:
Ko = Ko_num / Ko_den
display(Math(r"K_{nc} = "))
display(Ko)
display(Latex(r"Учтем задержку в вычислительном устройстве и домножим ПФ на $\frac{1}{z}$"))
Ko *= 1/z
display(Math(r"K_{nc} = "))
display(Ko)

<IPython.core.display.Math object>

(0.255*z + 0.134)/(25.0*z**2 - 28.83*z + 3.38)

<IPython.core.display.Latex object>

<IPython.core.display.Math object>

(0.255*z + 0.134)/(z*(25.0*z**2 - 28.83*z + 3.38))

In [133]:
B, A = smp.fraction(Ko)
B = smp.Poly(B)
A = smp.Poly(A)

In [135]:
j = 0

Чтобы у полиномиального уравнения существовало решение, необходимо выбирать порядки полиномов следующим образом:
$$
\begin{align}
&\deg D_з(z) = 2 \cdot \deg A(z) - 1 = 2 \cdot 4 - 1 = 7 \\
&\deg Q_р(z) = \deg A(z) - 1 = 4 - 1 = 3 \\
&\deg C_р(z) = \deg A(z) - 1 = 4 - 1 = 3
\end{align}
$$
Полученный полином замкнутой системы задается своими корнями.
$$D_з(z) = \prod_{i=1}^n (z - z_i) \qquad |z_i| < 1,\quad i = 1,2,...,n$$

In [136]:
deg_A = smp.degree(A)
deg_B = smp.degree(B)
deg_D = 2 * deg_A +j - 1
deg_Q = deg_A - 1 
deg_C = deg_A + j - 1

In [78]:
deg_C

3

Зададим корни желаемого полинома

In [191]:
roots = [0.5, 0.5, 0.5, 0.5,0.5]
assert len(roots) == deg_D
D = 1
for r in roots:
    D *= (z-r)
D = smp.Poly(D)
display(Math(r"D(z) = "))
display(D)

<IPython.core.display.Math object>

Poly(1.0*z**5 - 2.5*z**4 + 2.5*z**3 - 1.25*z**2 + 0.3125*z - 0.03125, z, domain='RR')

In [192]:
As = A.all_coeffs()
Bs = B.all_coeffs()
Ds = D.all_coeffs()
while len(Bs) < len(As):
    Bs.insert(0, 0)

In [193]:
As[0]

25.0000000000000

In [194]:
LMat = np.array([[As[0],0,0,Bs[0],0,0],[As[1],As[0],0,Bs[1],Bs[0],0],[As[2],As[1],As[0],Bs[2],Bs[1],Bs[0]],[As[3],As[2],As[1],Bs[3],Bs[2],Bs[1]],[0,As[3],As[2],0,Bs[3],Bs[2]],[0,0,As[3],0,0,Bs[3]]], dtype=float)

In [107]:
LMat = np.zeros((len(Ds), len(Ds)), dtype=float)
#LMat = smp.zeros(len(Ds), len(Ds))
half = deg_D//2
half = half if half%2==0 else half+1
LMat = np.array([[As[0],0,0,Bs[0],0,0,0],[As[1] - As[0],As[0],0,Bs[1],Bs[0],0,0],[As[2]-As[1],As[1]-As[0],As[0],Bs[2],Bs[1],Bs[0],0],[As[3]-As[2],As[2]-As[1],As[1]-As[0],Bs[3],Bs[2],Bs[1],Bs[0]],[-As[3],As[3]-As[2],As[2]-As[1],0,Bs[3],Bs[2],Bs[1]],[0,-As[3],As[3]-As[2],0,0,Bs[3],Bs[2]],[0,0,-As[3],0,0,0,Bs[3]]], dtype=float)
# for i in range(LMat.shape[0]):
#     for j in range(i, -1, -1):
#         print(i,j)
#         if j >= half:
#             continue
#         if not i-j >= len(As):
#             LMat[i][j] = As[i-j]
#         if not i-j >= len(Bs):
#             LMat[i][j+half] = Bs[i-j]

In [195]:
LMat

array([[ 25.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ],
       [-28.83 ,  25.   ,   0.   ,   0.   ,   0.   ,   0.   ],
       [  3.38 , -28.83 ,  25.   ,   0.255,   0.   ,   0.   ],
       [  0.   ,   3.38 , -28.83 ,   0.134,   0.255,   0.   ],
       [  0.   ,   0.   ,   3.38 ,   0.   ,   0.134,   0.255],
       [  0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.134]])

In [196]:
Ds

[1.00000000000000,
 -2.50000000000000,
 2.50000000000000,
 -1.25000000000000,
 0.312500000000000,
 -0.0312500000000000]

In [197]:
QCs = np.linalg.solve(LMat, np.array(Ds, dtype=float))

In [198]:
Qs = QCs[:len(QCs)//2]
Cs = QCs[len(QCs)//2:]
Q = smp.Poly(Qs, z)
C = smp.Poly(Cs, z)
display(Math(r"Q(z) ="))
display(Q)
display(Math(r"C(z) ="))
display(C)

<IPython.core.display.Math object>

Poly(0.04*z**2 - 0.053872*z + 0.0455024238508839, z, domain='RR')

<IPython.core.display.Math object>

Poly(-1.27800139714548*z**2 + 1.62813500720972*z - 0.233208955223881, z, domain='RR')

In [199]:
Kc = C/(Q)
display(Math(r"K_{c} = "))
display(Kc)

<IPython.core.display.Math object>

(-1.27800139714548*z**2 + 1.62813500720972*z - 0.233208955223881)/(0.04*z**2 - 0.053872*z + 0.0455024238508839)

In [200]:
def print_matlab_coeffs(arr):
    print("[", end="")
    for a in arr:
        print(a, end=" ")
    print("]")

def print_matlab_coeffs_tf(tf):
    num, den = smp.fraction(tf)
    num = smp.Poly(num, z).all_coeffs()
    den = smp.Poly(den, z).all_coeffs()
    print_matlab_coeffs(num)
    print_matlab_coeffs(den)

In [201]:
print_matlab_coeffs(Cs)
print_matlab_coeffs(Qs)

[-1.2780013971454816 1.6281350072097163 -0.23320895522388058 ]
[0.03999999999999999 -0.05387200000000002 0.045502423850883905 ]


In [155]:
Q*(z-1)

Poly(0.04*z**3 - 0.093872*z**2 + 0.0993744238508839*z - 0.0455024238508839, z, domain='RR')

In [47]:
W = Kc * Ko
Phi = smp.simplify(W / (1 + W))
num_Phi, den_Phi = smp.fraction(Phi)
smp.roots(den_Phi)

{-0.749999999999998: 1,
 -0.500000000000002: 1,
 -0.249999999999999: 1,
 0.100000000000000: 1,
 0.300000000000002: 1,
 0.599999999999992: 1,
 0.800000000000005: 1}

In [48]:
smp.roots(num_Phi)

{-1.45134725204630: 1,
 0.00303544923237146: 1,
 0.233805890691113: 1,
 0.799029237922506 - 0.0527990613901999*I: 1,
 0.799029237922506 + 0.0527990613901999*I: 1}

In [51]:
print_matlab_coeffs_tf(W)

[2.04393216242594 -0.783955420972727 -3.35745561092059 2.71433211545817 -0.452952973914017 0.00135000000000000 ]
[1.00000000000000 -0.299999999999999 -2.98643216242594 1.01120542097273 3.58773061092059 -2.75329461545817 0.440915473914017 0.0 ]
